In [ ]:
import math

import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams['figure.figsize'] = [14, 9]

In [ ]:
def paretoICDF(n=1000, a=1.2, lb=300*1024, ub=1*1024**3):
    u = np.linspace(0, 1, n)
    lb, ub = np.power(lb, a), np.power(ub, a)
    x = - (u*ub - u*lb - ub) / (ub*lb)
    x = np.power(x, -1/a)
    x = np.round(x, 0).astype(int)
    return x

In [ ]:
n = 100000
fig, [ax0, ax1] = plt.subplots(1, 2)
for a in [1.01, 1.05, 1.1, 1.2, 1.3, 1.4]:
    x = paretoICDF(a=a, n=n)
    ax0.plot(x, np.cumsum(x)/np.sum(x), label=f'a={a}')
    ax1.bar(str(a), np.mean(x))
ax0.legend()
ax0.set_ylabel('cumulative probability')
ax0.set_xlabel('x')
ax0.set_title('CDF')
ax1.set_xlabel('alpha')
ax1.set_ylabel('x')
ax1.set_title('Mean of distribution')
fig.suptitle(f'Pareto CDF and mean, n={n} samples of x with equal probability')
fig.tight_layout()

In [ ]:
def paretoCDF(n=1000, a=1.2, lb=300*1024, ub=1*1024**3, x=None):
    if x is None:
        x = np.linspace(lb, ub, n)
    x = np.round(x, 0).astype(int)
    p = (1 - np.power(lb, a)*np.power(x, -a)) / (1 - np.power(lb/ub, a))
    return x, p

In [ ]:
def discretize(x, cdf):
    xx = np.convolve(x, [.5,.5], 'valid')
    pp = np.convolve(cdf, [1, -1], 'valid')
    xx = np.round(xx, 0).astype(int)
    return xx, pp

In [ ]:
n = 250
fig, [ax0, ax1] = plt.subplots(1, 2)
for a in [0.8, 1.01, 1.05, 1.1, 1.2, 1.3, 1.4]:
    x = np.logspace(math.log(300*1024, 1024), 3, n, base=1024)
    x, p = paretoCDF(a=a, x=x)
    x, p = discretize(x, p)

    ax0.plot(x, np.cumsum(p*x)/np.sum(p*x), label=f'a={a}')
    ax0.axvline(x=55*1024**2, color='red', linestyle='--')
    ax1.bar(str(a), np.mean(p*x))

ax0.legend()
ax0.legend()
ax0.set_ylabel('cumulative probability')
ax0.set_xlabel('x')
ax0.set_title('CDF')
ax1.set_xlabel('alpha')
ax1.set_ylabel('x')
ax1.set_title('Mean of distribution')
fig.suptitle(f'Pareto CDF and mean, n={n} samples exponentially spaced over x and discretized probability')

In [ ]:
# with open('../tests/traffic.cue', 'w') as file:
#     x = paretoICDF(a=1.2, n=1000)
#     file.write(f'package tests\n\n_connsizespareto: [{",".join(map(str,x))}]')

In [ ]:
with open('../tests/traffic.cue', 'w') as file:
    n = 250
    x = np.logspace(math.log(300*1024, 1024), 3, n, base=1024)
    x, p = paretoCDF(a=1.2, x=x)
    x, p = discretize(x, p)
    file.write(f'package tests\n\n_connsizespareto: [{",".join(map(str,x))}]\n_connprobpareto: [{",".join(map(str,p))}]')